In [ ]:
import boto3
import py7zr
import configparser
from pathlib import Path
from pyunpack import Archive
import os
import zipfile 
from datetime import * 
from io import BytesIO 
import json 
import re 
import pickle
import tempfile

In [ ]:
# Create a ConfigParser instance to read configuration from 'credentials.txt'
config = configparser.ConfigParser()
config.read_file(open('credentials.txt'))

In [ ]:
# Retrieve AWS access key and secret from the configuration file
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

In [ ]:
# Create an AWS session using the retrieved access key and secret
session = boto3.Session(
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)

In [ ]:
s3_client = session.client('s3')
s3_resource = session.resource('s3')

In [ ]:
# Name of the S3 bucket
bucket_name = 'caged-bucket'

# Paths for different file operations
zip_folder = 'raw/'
unzipped_folder = 'raw/uncompressed/'

# List to store names of compacted files
compacted_files = []

# List objects in the specified S3 bucket and prefix
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=zip_folder)

# Iterate through the S3 objects
for obj in response.get('Contents', []):
    # Check if the object has the extension ".7z"
    if obj['Key'].lower().endswith('.7z'):
        # Get only the file name without the prefix
        compacted_files.append(os.path.basename(obj['Key']))

# Iterate through the compacted files
for f in compacted_files:
    zip_file = f

    # Create a temporary directory for extracting files
    temp_dir = tempfile.mkdtemp()

    # Get the .7z object from S3
    zip_obj = s3_client.get_object(Bucket=bucket_name, Key=f'{zip_folder}{zip_file}')

    # Save the .7z file locally
    local_file_path = os.path.join(temp_dir, zip_file)
    with open(local_file_path, 'wb') as f:
        f.write(zip_obj['Body'].read())

    # Extract the .7z files
    extracted_files = Archive(local_file_path).extractall(temp_dir)

    # Upload the extracted files to S3
    for root, _, files in os.walk(temp_dir):
        for file in files:
            file_path = os.path.join(root, file)
            s3_key = os.path.relpath(file_path, temp_dir).replace("\\", "/")
            s3_client.upload_file(
                Filename=file_path,
                Bucket=bucket_name,
                Key=f'{unzipped_folder}/{s3_key}',
                ExtraArgs={'ContentType': 'text/plain'}
            )

            print(f"File {file} copied to {bucket_name}/{unzipped_folder}/{s3_key}")
    
    # Remove the locally saved .7z file
    os.remove(local_file_path)

    print(f"Decompression of file {zip_file} completed")

Arquivo CAGEDEST_042016.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042016.txt
Descompactação do arquivo CAGEDEST_042016.7z concluída com sucesso
Arquivo CAGEDEST_042017.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042017.7z
Arquivo CAGEDEST_042017.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042017.txt
Descompactação do arquivo CAGEDEST_042017.7z concluída com sucesso
Arquivo CAGEDEST_042018.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042018.7z
Arquivo CAGEDEST_042018.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042018.txt
Descompactação do arquivo CAGEDEST_042018.7z concluída com sucesso
Arquivo CAGEDEST_042019.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042019.7z
Arquivo CAGEDEST_042019.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_042019.txt
Descompactação do arquivo CAGEDEST_042019.7z concluída com sucesso
Arquivo CA

Arquivo CAGEDEST_072017.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_072017.7z
Arquivo CAGEDEST_072017.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_072017.txt
Descompactação do arquivo CAGEDEST_072017.7z concluída com sucesso
Arquivo CAGEDEST_072018.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_072018.7z
Arquivo CAGEDEST_072018.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_072018.txt
Descompactação do arquivo CAGEDEST_072018.7z concluída com sucesso
Arquivo CAGEDEST_072019.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_072019.7z
Arquivo CAGEDEST_072019.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_072019.txt
Descompactação do arquivo CAGEDEST_072019.7z concluída com sucesso
Arquivo CAGEDEST_082010.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_082010.7z
Arquivo CAGEDEST_082010.txt copiado com sucesso para caged-bucket/raw/uncompres

Arquivo CAGEDEST_102017.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_102017.txt
Descompactação do arquivo CAGEDEST_102017.7z concluída com sucesso
Arquivo CAGEDEST_102018.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_102018.7z
Arquivo CAGEDEST_102018.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_102018.txt
Descompactação do arquivo CAGEDEST_102018.7z concluída com sucesso
Arquivo CAGEDEST_102019.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_102019.7z
Arquivo CAGEDEST_102019.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_102019.txt
Descompactação do arquivo CAGEDEST_102019.7z concluída com sucesso
Arquivo CAGEDEST_112010.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_112010.7z
Arquivo CAGEDEST_112010.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDEST_112010.txt
Descompactação do arquivo CAGEDEST_112010.7z concluída com sucesso
Arquivo CA

Arquivo CAGEDMOV202009.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202009.7z
Arquivo CAGEDMOV202009.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202009.txt
Descompactação do arquivo CAGEDMOV202009.7z concluída com sucesso
Arquivo CAGEDMOV202010.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202010.7z
Arquivo CAGEDMOV202010.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202010.txt
Descompactação do arquivo CAGEDMOV202010.7z concluída com sucesso
Arquivo CAGEDMOV202011.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202011.7z
Arquivo CAGEDMOV202011.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202011.txt
Descompactação do arquivo CAGEDMOV202011.7z concluída com sucesso
Arquivo CAGEDMOV202012.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202012.7z
Arquivo CAGEDMOV202012.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV20201

Arquivo CAGEDMOV202304.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202304.7z
Arquivo CAGEDMOV202304.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202304.txt
Descompactação do arquivo CAGEDMOV202304.7z concluída com sucesso
Arquivo CAGEDMOV202305.7z copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202305.7z
Arquivo CAGEDMOV202305.txt copiado com sucesso para caged-bucket/raw/uncompressed//CAGEDMOV202305.txt
Descompactação do arquivo CAGEDMOV202305.7z concluída com sucesso


In [ ]:
directory_to_delete = 'raw/uncompressed/'

# Listing all the objects in the bucket
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=directory_to_delete)

# Iterate through the objects
for obj in response.get('Contents', []):
    # Checking if the object is ".7z" file
    if obj['Key'].lower().endswith('.7z'):
        # Delete the object
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        print(f"Arquivo {obj['Key']} excluído com sucesso")

Arquivo raw/uncompressed//CAGEDMOV202012.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202101.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202102.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202103.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202104.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202105.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202106.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202107.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202108.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202109.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202110.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202111.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202112.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202201.7z excluído com sucesso
Arquivo raw/uncompressed//CAGEDMOV202202.7z excluído com sucesso
Arquivo raw/uncompressed/